# SPML HW4: Model Extraction

In this notebook you'll explore model extraction.

In [ ]:
######### Make sure to put your info #########
name = 'Alireza Farajtabrizi'
std_id = '403206554'
##############################################

In [ ]:
import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, ConcatDataset

from tqdm import trange
import matplotlib.pyplot as plt

import torchvision
from torchvision import transforms, datasets, models

torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Loading CIFAR100 (5 points)

Load the `CIFAR100` dataset. Make sure you resize the images to be `224x224` (same as the input size of resnet).

In [ ]:
# TODO: Load CIFAR-100 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),])

# Load the CIFAR-100 dataset
cifar100_trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
cifar100_testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

cifar100_trainloader = torch.utils.data.DataLoader(cifar100_trainset, batch_size=128, shuffle=True, num_workers=2)
cifar100_testloader = torch.utils.data.DataLoader(cifar100_testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# Pre-trained ResNet34 (10 points)

Load a pre-trained ResNet34 and train it on the `CIFAR100` dataset.

In [ ]:
# TODO: Load pretrained ResNet-34 model
resnet34_victim = models.resnet34(weights='IMAGENET1K_V1')
num_ftrs = resnet34_victim.fc.in_features
resnet34_victim.fc = nn.Linear(num_ftrs, 100)
resnet34_victim = resnet34_victim.to(device)

In [ ]:
# TDOO: Train the model on CIFAR100
n_epochs = 10
learning_rate = 1e-3

# Set the model to training mode
resnet34_victim.train()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet34_victim.parameters(), lr=learning_rate)

# Train the model
for epoch in trange(n_epochs):
    resnet34_victim.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in cifar100_trainloader:
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet34_victim(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

        # Compute accuracy
        predicted = outputs.argmax(dim = 1)  # Get the class with the highest score
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    # Calculate average loss and accuracy for the epoch
    epoch_loss = running_loss / len(cifar100_trainloader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

 10%|█         | 1/10 [04:13<37:58, 253.15s/it]

Epoch 1/10, Loss: 1.6262, Accuracy: 53.8840


 20%|██        | 2/10 [08:25<33:40, 252.54s/it]

Epoch 2/10, Loss: 1.0687, Accuracy: 68.1480


 30%|███       | 3/10 [12:37<29:26, 252.31s/it]

...

Epoch 8/10, Loss: 0.2026, Accuracy: 93.3640


 90%|█████████ | 9/10 [37:50<04:12, 252.23s/it]

Epoch 9/10, Loss: 0.1399, Accuracy: 95.4900


100%|██████████| 10/10 [42:02<00:00, 252.21s/it]

Epoch 10/10, Loss: 0.1456, Accuracy: 95.1900


In [ ]:
# Evaluate the model on the test set
def test_model(model, loader):
    model.to(device)
    model.eval()  # Set the model to evaluation mode

    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for testing
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)

            # Get predictions
            predicted = outputs.argmax(dim = 1) # Get the class with the highest score

            # Update total and correct counts
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    return accuracy

print(f'Accuracy of the model on the test set: {test_model(resnet34_victim, cifar100_testloader)}%')

Accuracy of the model on the test set: 68.65%


You might want to save this model to avoid retraining.

In [ ]:
torch.save(resnet34_victim.state_dict(), 'resnet34_cifar100trainset.pth')

# Model Extraction (20 points)

Here we use knowledge distillation to extract models. If you are confused after the instructions take a look at the next section to understand what we are trying to do. The general steps in Knowledge Distillation are as follows:

1. Set the victim (teacher) to evaluation mode and the attacker (student) to training mode.
2. Use the victim to find the logits for each batch of inputs.
3. Predict the attackers output for the same batch of inputs.
4. Define and reduce the loss function over the difference between logits from the victim and attacker (use KL-Divergence, ...)
5. Repeat steps 2-4 for the number of epochs.

Feel free to check out [Distilling the Knowledge in a Neural Network](https://arxiv.org/abs/1503.02531) to get a better sense of the process.

In [ ]:
def knowledge_distillation(victim_model, attacker_model, loader, optimizer, epochs, T):
    attacker_model.train()
    victim_model.eval()
    for epoch in trange(n_epochs):

        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, _ in loader:
            inputs = inputs.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                teacher_outputs = victim_model(inputs) / T
                teacher_probs = F.softmax(teacher_outputs, dim=1)
                labels = teacher_probs.argmax(dim = 1)

            student_outputs = attacker_model(inputs) / T
            student_log_probs = F.log_softmax(student_outputs, dim=1)

            distillation_loss = F.kl_div(student_log_probs, teacher_probs, reduction='batchmean') * (T**2)
            #classification_loss = F.cross_entropy(student_outputs, labels)
            #total_loss = 0.9 * distillation_loss + 0.1 * classification_loss

            distillation_loss.backward()
            optimizer.step()

            running_loss += distillation_loss.item()
            predictions = student_outputs.argmax(dim = 1)
            correct_preds += (predictions == labels).sum().item()
            total_preds += labels.size(0)

        epoch_loss = running_loss / len(loader)
        epoch_accuracy = correct_preds / total_preds * 100
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

Can you explain how we should set the temperature? Why is this choice appropriate for model extraction?

`your response:`
##### some number greater than 1 but not too big I chose 10.
##### It impact the dominance of logit vector and it makes it more smooth. Knowledge distilliation is pointless without soft labels and higher T makes labels softer.

# Attack Transferability (20 points)

Implement attacks such as FGSM or PGD, you can use code from previous homeworks or readily available libraries.

In [ ]:
!pip install torchattacks

In [ ]:
# TODO: Load or implement attacks
from torchattacks import PGD

Fill in the following function to attack a model and report the accuracy of the victim on the adversarial examples generated using the available model.

In [ ]:
def transferability_attack(model, victim, loader, attack):
    model.eval()
    victim.eval()

    correct = 0
    total = 0

    for inputs, labels in tqdm.tqdm(loader):
        inputs, labels = inputs.to(device), labels.to(device)
        adv_inputs = attack(inputs, labels)

        # Evaluate the victim model on adversarial examples
        with torch.no_grad():
            outputs = victim(adv_inputs)
            predicted = outputs.argmax(dim = 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy

# CIFAR10 (35 points)

## Loading and Exploration (5 points)

First load the `CIFAR10` dataset.

In [31]:
# TODO: Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),])

cifar10_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar10_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

cifar10_trainloader = torch.utils.data.DataLoader(cifar10_trainset, batch_size=128, shuffle=True, num_workers=2)
cifar10_testloader = torch.utils.data.DataLoader(cifar10_testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


Which classes from the `CIFAR10` dataset are present in `CIFAR100` classes?

In [33]:
# TODO: Check if classes are present in both datasets
print(cifar100_trainset.classes)
print(cifar10_trainset.classes)
print(set(cifar100_trainset.classes).intersection(set(cifar10_trainset.classes)))

['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree',

Now use the test dataset from `CIFAR10` to extract the model.

## Pre-trained ResNet18 (10 points)

Use the pre-trained ResNet18 dataset and extract the model using knowledge distillation.

In [20]:
# TODO: Load pretrained ResNet-18 model
resnet18_pretrained = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = resnet18_pretrained.fc.in_features
resnet18_pretrained.fc = nn.Linear(num_ftrs, 100)
resnet18_pretrained = resnet18_pretrained.to(device)

# TODO: Extract the model
n_epochs = 10
learning_rate = 1e-3
temperature = 10

optimizer = optim.Adam(resnet18_pretrained.parameters(), lr=learning_rate)
knowledge_distillation(resnet34_victim, resnet18_pretrained, cifar10_testloader, optimizer, n_epochs, T=temperature)
torch.save(resnet18_pretrained.state_dict(), 'resnet18_pretrained_cifar10_testset.pth')

 10%|█         | 1/10 [00:48<07:16, 48.47s/it]

Epoch 1/10, Loss: 4.7813, Accuracy: 28.4400


 20%|██        | 2/10 [01:35<06:20, 47.55s/it]

Epoch 2/10, Loss: 2.9988, Accuracy: 40.3600


 30%|███       | 3/10 [02:22<05:31, 47.38s/it]

...

Epoch 8/10, Loss: 1.0937, Accuracy: 68.1700


 90%|█████████ | 9/10 [07:06<00:47, 47.29s/it]

Epoch 9/10, Loss: 1.0006, Accuracy: 69.3600


100%|██████████| 10/10 [07:53<00:00, 47.38s/it]

Epoch 10/10, Loss: 0.9112, Accuracy: 70.9400


What is the accuracy of the extracted model on the `CIFAR100` test set?

In [21]:
# TODO: Report accuracy on CIFAR100
clean_accuracy = test_model(resnet18_pretrained, cifar100_testloader)
print(f'Accuracy of the model on the test set: {clean_accuracy}%')

Accuracy of the model on the test set: 38.61%


In [22]:
attack = PGD(resnet18_pretrained,eps=8/255, alpha=2/255, steps=10)
adv_acurracy = transferability_attack(resnet18_pretrained, resnet34_victim, cifar100_testloader, attack)
print(f'\nAccuracy of the model on Transfered Adversarial Examples from test set: {adv_acurracy}%')

100%|██████████| 79/79 [03:53<00:00,  2.96s/it]


Accuracy of the model on Transfered Adversarial Examples from test set: 7.59%


## ResNet18 (10 points)

Repeat the pervious steps but without pre-training.

In [23]:
# TODO: Load ResNet-18 model
resnet18_random = models.resnet18()
num_ftrs = resnet18_random.fc.in_features
resnet18_random.fc = nn.Linear(num_ftrs, 100)
resnet18_random = resnet18_random.to(device)

# TODO: Extract the model
n_epochs = 20
learning_rate = 1e-3
temperature = 10

optimizer = optim.Adam(resnet18_random.parameters(), lr=learning_rate)
knowledge_distillation(resnet34_victim, resnet18_random, cifar10_testloader, optimizer, n_epochs, T=temperature)
torch.save(resnet18_random.state_dict(), 'resnet18_random_cifar10_testset.pth')

  5%|▌         | 1/20 [00:47<14:56, 47.20s/it]

Epoch 1/20, Loss: 7.2948, Accuracy: 13.9800


 10%|█         | 2/20 [01:34<14:12, 47.35s/it]

Epoch 2/20, Loss: 6.2510, Accuracy: 18.1300


 15%|█▌        | 3/20 [02:22<13:24, 47.35s/it]

...

Epoch 18/20, Loss: 1.6494, Accuracy: 58.9900


 95%|█████████▌| 19/20 [15:01<00:47, 47.46s/it]

Epoch 19/20, Loss: 1.5546, Accuracy: 60.7200


100%|██████████| 20/20 [15:49<00:00, 47.45s/it]

Epoch 20/20, Loss: 1.4699, Accuracy: 61.6900


Measure the accuracy of the newly distillied attacker and compare your results from the previous section.

In [32]:
# TODO: Report accuracy on CIFAR100
clean_accuracy = test_model(resnet18_random, cifar100_testloader)
print(f'Accuracy of the model on the test set: {clean_accuracy}%')

Accuracy of the model on the test set: 18.14%


In [24]:
attack = PGD(resnet18_random, eps=8/255, alpha=2/255, steps=10)
adv_acurracy = transferability_attack(resnet18_random, resnet34_victim, cifar100_testloader, attack)
print(f'\nAccuracy of the model on Transfered Adversarial Examples from test set: {adv_acurracy}%')

100%|██████████| 79/79 [03:55<00:00,  2.97s/it]


Accuracy of the model on Transfered Adversarial Examples from test set: 34.72%


What are the effects of pre-training?

`your response:`
##### The pretraining effects the training of the extracted model by reducing the clean accuracy and therefore the fidelty of extracted model.
##### It seems that same strating point (same dataset they were trained on) end up to better fidelty

## Full Dataset (10 points)

Repeat your experiments using the pre-trained ResNet18 but this time use the entire CIFAR10 dataset.

In [25]:
# TODO: Load pretrained ResNet-18 model
cifar10_full_dataset = ConcatDataset([cifar10_trainset, cifar10_testset])
cifar10_full_dataloader = torch.utils.data.DataLoader(cifar10_full_dataset, batch_size=128, shuffle=True, num_workers=2)

resnet18_pretrained2 = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = resnet18_pretrained2.fc.in_features
resnet18_pretrained2.fc = nn.Linear(num_ftrs, 100)
resnet18_pretrained2 = resnet18_pretrained2.to(device)

# TODO: Extract the model
n_epochs = 10
learning_rate = 1e-3
temperature = 10

optimizer = optim.Adam(resnet18_pretrained2.parameters(), lr=learning_rate)
knowledge_distillation(resnet34_victim, resnet18_pretrained2, cifar10_full_dataloader, optimizer, n_epochs, T=temperature)
torch.save(resnet18_pretrained2.state_dict(), 'resnet18_pretrained_cifar10_fullset.pth')

 10%|█         | 1/10 [04:41<42:12, 281.35s/it]

Epoch 1/10, Loss: 2.9456, Accuracy: 42.0567


 20%|██        | 2/10 [09:23<37:33, 281.67s/it]

Epoch 2/10, Loss: 1.7162, Accuracy: 54.3450


 30%|███       | 3/10 [14:04<32:50, 281.46s/it]

...

Epoch 8/10, Loss: 0.5902, Accuracy: 75.4100


 90%|█████████ | 9/10 [42:13<04:41, 281.47s/it]

Epoch 9/10, Loss: 0.5583, Accuracy: 76.1833


100%|██████████| 10/10 [46:55<00:00, 281.53s/it]

Epoch 10/10, Loss: 0.5151, Accuracy: 77.1017


Report the accuracy on the `CIFAR100` testset once more.

In [26]:
# TODO: Report accuracy on CIFAR100
clean_accuracy = test_model(resnet18_pretrained2, cifar100_testloader)
print(f'Accuracy of the model on the test set: {clean_accuracy}%')

Accuracy of the model on the test set: 58.05%


In [27]:
attack = PGD(resnet18_pretrained2, eps=8/255, alpha=2/255, steps=10)
adv_acurracy = transferability_attack(resnet18_pretrained2, resnet34_victim, cifar100_testloader, attack)
print(f'\nAccuracy of the model on Transfered Adversarial Examples from test set: {adv_acurracy}%')

100%|██████████| 79/79 [03:53<00:00,  2.96s/it]


Accuracy of the model on Transfered Adversarial Examples from test set: 0.38%


What are the effects of using more data?

`your response:`
##### Use of more data for extraction had huge Impact on Clean (10% difference only) Accuracy and Therefore the fidelty. It has better accuracy than the first pretrained one and as a result the adversarial examples attacks are more successful.

# CIFAR100 (10 points)

This time, use the training dataset from `CIFAR100` and perform knowledge distillation on a pre-trained ResNet18.

In [28]:
# TODO: Load pretrained ResNet-18 model
resnet18_pretrained3 = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = resnet18_pretrained2.fc.in_features
resnet18_pretrained3.fc = nn.Linear(num_ftrs, 100)
resnet18_pretrained3 = resnet18_pretrained3.to(device)

# TODO: Extract the model
n_epochs = 10
learning_rate = 1e-3
temperature = 10

optimizer = optim.Adam(resnet18_pretrained3.parameters(), lr=learning_rate)
knowledge_distillation(resnet34_victim, resnet18_pretrained3, cifar100_trainloader, optimizer, n_epochs, T=temperature)
torch.save(resnet18_pretrained3.state_dict(), 'resnet18_pretrained_cifar100_trainset.pth')

 10%|█         | 1/10 [03:54<35:12, 234.76s/it]

Epoch 1/10, Loss: 5.8103, Accuracy: 49.0580


 20%|██        | 2/10 [07:49<31:16, 234.52s/it]

Epoch 2/10, Loss: 2.9678, Accuracy: 70.2840


 30%|███       | 3/10 [11:44<27:23, 234.73s/it]

...

Epoch 8/10, Loss: 0.9479, Accuracy: 91.6140


 90%|█████████ | 9/10 [35:12<03:54, 234.74s/it]

Epoch 9/10, Loss: 0.8632, Accuracy: 92.2860


100%|██████████| 10/10 [39:07<00:00, 234.72s/it]

Epoch 10/10, Loss: 0.8096, Accuracy: 93.0920


How does the accuracy change now?

In [29]:
# TODO: Report accuracy on CIFAR100
clean_accuracy = test_model(resnet18_pretrained3, cifar100_testloader)
print(f'Accuracy of the model on the test set: {clean_accuracy}%')

Accuracy of the model on the test set: 70.54%


In [34]:
attack = PGD(resnet18_pretrained3, eps=8/255, alpha=2/255, steps=10)
adv_acurracy = transferability_attack(resnet18_pretrained3, resnet34_victim, cifar100_testloader, attack)
print(f'\nAccuracy of the model on Transfered Adversarial Examples from test set: {adv_acurracy}%')

100%|██████████| 79/79 [03:54<00:00,  2.96s/it]


Accuracy of the model on Transfered Adversarial Examples from test set: 1.46%


Why do you suppose using the `CIFAR100` had the following results? Explain your observations.

`your response:`
##### What we did in this part is equal to normal Knowledge Distillation. Therefore we expected the result on clean accuracy to be the same and the attacks success rate be close to 100 which it was.
##### The clean Accuracy of the Extracted model is higher which can be due to overfit of the victim

##### چندتا چیزو هم فارسی توضیح بدم:اون دقت مدلی که تو تقطیر دانش پرینت کردم مقایسه آرگمکسه مدل مهاجم و مدل قربانیه و کاری به لیبل اصلی نداشتم.
##### روال کلی نتایج هم خیلی خوب و معنا داره مدل پری تریند اول خوب یاد میگیره و دقتش رو داده های تست 30 درصد فاصله داره و اتک هاش 68 رو میکنه 8 درصد مدل بدون پری تریند دقت عادیش خیلی بده سر همین اتک هاش خوب ترنسفر نمیشن و 68 رو می کنن 34 که عملا خیلی ضعیفه
##### بعد که داده های ترین سیفار 100 رو استفاده میکنیم خیلی بهتر میشه سرقت و افزایش داده اثرش رو نشون میده و فاصله در دقت عادی میشه 18 درصد و اتک ها هم خیلی قوی ترنسفر میشن و 68 رو میارن زیر 1 درصد
##### در نهایت استفاده از همون داده های اصلی آموزش قربانی برای مدل مهاجم استفاده میشه که دقت عادی رو میچسبونه به مدل قربانی و اتک هاش هم 68 رو میاره رو 1 درصد. 2 نکته وجود داره:
##### اینکار این مرحله عملا همون تقطیر دانش عادیه چون دیتاست ها یکیه فقط لاسمون ترم دوم رو نداره.
##### مدل مهاجم دقتش بهتره چون احتمالا بخاطر ظرفیت بیشتر مدل قربانی اورفیت بیشتر داشته.